# Create Combined All Data - Fixed Quarter Dates

In [73]:
# Whether this is a sample/debug run or not
DEBUG = False

In [74]:
# Packages
import pandas as pd
import openpyxl

In [75]:
# Load ~\Box\STAT 222 Capstone\Intermediate Data\Credit_Rating\credit_ratings_on_fixed_quarter_dates_with_earnings_call_date.csv
cr_and_dates = pd.read_csv(r'~\Box\STAT 222 Capstone\Intermediate Data\Credit_Rating\credit_ratings_on_fixed_quarter_dates_with_earnings_call_date.csv')
print(cr_and_dates)

     ticker          fixed_quarter_date earnings_call_date Rating  \
0      AAPL  2014-07-01 00:00:00.000000         2014-04-23     AA   
1      AAPL  2014-10-01 00:00:00.000000         2014-07-22     AA   
2      AAPL  2015-01-01 00:00:00.000000         2014-10-20     AA   
3      AAPL  2015-04-01 00:00:00.000000         2015-01-27     AA   
4      AAPL  2015-07-01 00:00:00.000000         2015-04-27     AA   
...     ...                         ...                ...    ...   
7976    ZTS  2015-10-01 00:00:00.000000         2015-08-04    BBB   
7977    ZTS  2016-01-01 00:00:00.000000         2015-11-03    BBB   
7978    ZTS  2016-04-01 00:00:00.000000         2016-02-16    BBB   
7979    ZTS  2016-07-01 00:00:00.000000         2016-05-04    BBB   
7980    ZTS  2016-10-01 00:00:00.000000         2016-08-03    BBB   

                      Rating Agency Name rating_date         Source  \
0     Standard & Poor's Ratings Services  2014-05-27  Supplementary   
1     Standard & Poor's Ratin

In [76]:
# drop source column (about to get it again via the next join)
cr_and_dates = cr_and_dates.drop(columns = ['source'])

In [77]:
# check for duplicates on ticker by fixed_quarter_date
print(cr_and_dates[cr_and_dates.duplicated(subset=['ticker', 'fixed_quarter_date'], keep=False)])

Empty DataFrame
Columns: [ticker, fixed_quarter_date, earnings_call_date, Rating, Rating Agency Name, rating_date, Source, Rating Rank AAA is 10, Next Rating, Next Rating Date, Previous Rating, Previous Rating Date, next_rating_date_or_end_of_data, credit_rating_year, previous_fixed_quarter_date, days_since_call]
Index: []


In [78]:
# Load ~\Box\STAT 222 Capstone\Intermediate Data\Calls\calls.csv
if DEBUG:
    calls = pd.read_parquet(r'~\Box\STAT 222 Capstone\Intermediate Data\Calls\combined_calls.parquet', nrows = 10)
else:
    calls = pd.read_parquet(r'~\Box\STAT 222 Capstone\Intermediate Data\Calls\combined_calls.parquet')
# Delete the first column (index)
#calls = calls.iloc[:,1:]
# rename symbol to company
calls = calls.rename(columns = {'symbol':'company'})
# Convert date to be just the date part
#calls['date'] = calls['date'].str[:10]
print(calls)

      company  quarter  year earnings_call_datetime  \
0           A        3  2011    2011-02-15 00:00:00   
1           A        4  2011    2011-05-13 00:00:00   
2           A        1  2012    2011-08-16 00:00:00   
3           A        2  2012    2011-11-16 00:00:00   
4           A        3  2012    2012-02-16 00:00:00   
...       ...      ...   ...                    ...   
31067     ZTS        4  2015    2016-02-16 16:13:08   
31068     ZTS        1  2016    2016-05-04 15:49:11   
31069     ZTS        2  2016    2016-08-03 16:27:41   
31070     ZTS        3  2016    2016-11-02 14:19:14   
31071     ZTS        4  2016    2017-02-16 15:19:17   

                                                 content  source    web  \
0      Operator: Good day, ladies and gentlemen, and ...  kaggle  False   
1      Operator: Good day, ladies and gentlemen, and ...  kaggle  False   
2      Operator: Good day, ladies and gentlemen, and ...  kaggle  False   
3      Operator: Good day, ladies and g

In [79]:
# check for duplicates on company by date
print(calls[calls.duplicated(subset=['company', 'date'], keep=False)])

Empty DataFrame
Columns: [company, quarter, year, earnings_call_datetime, content, source, web, date]
Index: []


### Earnings Call Data Merge

In [80]:
# convert earnings_call_date to date part
print(type(cr_and_dates['earnings_call_date'][0]))
print(type(calls['date'][0]))
cr_and_dates['earnings_call_date'] = pd.to_datetime(cr_and_dates['earnings_call_date']).dt.date
#calls['date'] = pd.to_datetime(calls['date'])
print(type(cr_and_dates['earnings_call_date'][0]))
print(type(calls['date'][0]))

# Inner join cr_and_dates (on columns ticker, earnings_call_date) with calls.csv (on columns company, date)
merged = pd.merge(cr_and_dates, calls, left_on=['ticker', 'earnings_call_date'], right_on=['company', 'date'], how='inner')

# Print range of dates
print('date range')
print(merged['date'].min())
print(merged['date'].max())

# Range of year, credit_rating_year
for variable in ['year', 'credit_rating_year']:
    print(variable)
    print(merged[variable].min())
    print(merged[variable].max())

# Print rows where year is greater than 2016
# This is due to inconsistencies in the earnings call data year variable
print('years greater than 2016')
print(merged[merged['year'] > 2016][['earnings_call_date', 'ticker', 'company', 'date', 'year', 'credit_rating_year']])

# Free up memory from calls
del calls

merged

<class 'str'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
date range
2010-05-03
2016-09-30
year
2010
2016
credit_rating_year
2010
2016
years greater than 2016
Empty DataFrame
Columns: [earnings_call_date, ticker, company, date, year, credit_rating_year]
Index: []


,ticker,fixed_quarter_date,earnings_call_date,Rating,Rating Agency Name,rating_date,Source,Rating Rank AAA is 10,Next Rating,Next Rating Date,...,previous_fixed_quarter_date,days_since_call,company,quarter,year,earnings_call_datetime,content,source,web,date
0,AAPL,2014-07-01 00:00:00.000000,2014-04-23,AA,Standard & Poor's Ratings Services,2014-05-27,Supplementary,9,AA,2015-02-18,...,2014-04-01 00:00:00.000000,69,AAPL,2,2014,2014-04-23 20:33:08,"Operator: Good day, everyone, and welcome to t...",web,True,2014-04-23
1,AAPL,2014-10-01 00:00:00.000000,2014-07-22,AA,Standard & Poor's Ratings Services,2014-05-27,Supplementary,9,AA,2015-02-18,...,2014-07-01 00:00:00.000000,71,AAPL,3,2014,2014-07-22 22:26:03,"Operator: Good day, everyone, and welcome to t...",web,True,2014-07-22
2,AAPL,2015-01-01 00:00:00.000000,2014-10-20,AA,Standard & Poor's Ratings Services,2014-05-27,Supplementary,9,AA,2015-02-18,...,2014-10-01 00:00:00.000000,73,AAPL,4,2014,2014-10-20 23:20:12,"Operator: Good day, everyone, and welcome to t...",web,True,2014-10-20
3,AAPL,2015-04-01 00:00:00.000000,2015-01-27,AA,Standard & Poor's Ratings Services,2015-02-18,Supplementary,9,AA,2015-05-28,...,2015-01-01 00:00:00.000000,64,AAPL,1,2015,2015-01-27 20:38:04,Operator: Good day ladies and gentlemen and we...,web,True,2015-01-27
4,AAPL,2015-07-01 00:00:00.000000,2015-04-27,AA,Standard & Poor's Ratings Services,2015-06-02,Supplementary,9,AA,2015-08-25,...,2015-04-01 00:00:00.000000,65,AAPL,2,2015,2015-04-27 20:48:04,"Operator: Good day everyone, and welcome to th...",web,True,2015-04-27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7976,ZTS,2015-10-01 00:00:00.000000,2015-08-04,BBB,Standard & Poor's Ratings Services,2015-01-30,Both,7,BBB,2015-11-03,...,2015-07-01 00:00:00.000000,58,ZTS,2,2015,2015-08-04 15:19:08,Operator: Welcome to the Second Quarter 2015 F...,web,True,2015-08-04
7977,ZTS,2016-01-01 00:00:00.000000,2015-11-03,BBB,Standard & Poor's Ratings Services,2015-11-03,Supplementary,7,BBB,2016-01-22,...,2015-10-01 00:00:00.000000,59,ZTS,3,2015,2015-11-03 14:45:11,Operator: Good day and welcome to the Third Qu...,web,True,2015-11-03
7978,ZTS,2016-04-01 00:00:00.000000,2016-02-16,BBB,Standard & Poor's Ratings Services,2016-01-22,Both,7,BBB,2016-12-23,...,2016-01-01 00:00:00.000000,45,ZTS,4,2015,2016-02-16 16:13:08,Operator: Good day and welcome to the fourth q...,web,True,2016-02-16
7979,ZTS,2016-07-01 00:00:00.000000,2016-05-04,BBB,Standard & Poor's Ratings Services,2016-01-22,Both,7,BBB,2016-12-23,...,2016-04-01 00:00:00.000000,58,ZTS,1,2016,2016-05-04 15:49:11,Operator: Good day and welcome to the First Qu...,web,True,2016-05-04


In [81]:
# check for duplicates on ticker by fixed_quarter_date
print(merged[merged.duplicated(subset=['ticker', 'fixed_quarter_date'], keep=False)])

Empty DataFrame
Columns: [ticker, fixed_quarter_date, earnings_call_date, Rating, Rating Agency Name, rating_date, Source, Rating Rank AAA is 10, Next Rating, Next Rating Date, Previous Rating, Previous Rating Date, next_rating_date_or_end_of_data, credit_rating_year, previous_fixed_quarter_date, days_since_call, company, quarter, year, earnings_call_datetime, content, source, web, date]
Index: []

[0 rows x 24 columns]


In [82]:
# check values of source
print(merged['source'].value_counts())

source
web    7981
Name: count, dtype: int64


In [83]:
# Rename content to transcript
merged = merged.rename(columns = {'content':'transcript'})

### Financial Statements Data Merge

In [84]:
# Load tabular financial statement data
df = pd.read_parquet(r'~\Box\STAT 222 Capstone\Intermediate Data\Tabular_Fin\combined_corrected_tabular_financial_statements_data.parquet')
print(df)

# Perform merge
merged = pd.merge(merged, df, left_on=['company', 'year', 'quarter'], right_on=['symbol', 'calendarYear', 'period'], how='inner')    
print(merged)

            date symbol reportedCurrency      cik fillingDate  \
0     2016-12-31    BEP              USD  1533232  2017-03-01   
1     2016-09-30    BEP              USD  1533232  2016-10-03   
2     2016-06-30    BEP              USD  1533232  2016-07-05   
3     2016-03-31    BEP              USD  1533232  2016-04-01   
4     2015-12-31    BEP              USD  1533232  2016-01-04   
...          ...    ...              ...      ...         ...   
20820 2011-03-31   YORW              USD   108985  2011-05-06   
20821 2010-12-31   YORW              USD   108985  2011-03-08   
20822 2010-09-30   YORW              USD   108985  2010-11-05   
20823 2010-06-30   YORW              USD   108985  2010-08-06   
20824 2010-03-31   YORW              USD   108985  2010-05-07   

              acceptedDate  calendarYear  period  cashAndCashEquivalents  \
0      2017-03-01 06:27:55          2016       4             223000000.0   
1      2016-10-03 06:45:16          2016       3             232000

In [85]:
# check for duplicates on ticker by fixed_quarter_date
print(merged[merged.duplicated(subset=['ticker', 'fixed_quarter_date'], keep=False)])

Empty DataFrame
Columns: [ticker, fixed_quarter_date, earnings_call_date, Rating, Rating Agency Name, rating_date, Source, Rating Rank AAA is 10, Next Rating, Next Rating Date, Previous Rating, Previous Rating Date, next_rating_date_or_end_of_data, credit_rating_year, previous_fixed_quarter_date, days_since_call, company, quarter, year, earnings_call_datetime, transcript, source, web, date_x, date_y, symbol, reportedCurrency, cik, fillingDate, acceptedDate, calendarYear, period, cashAndCashEquivalents, shortTermInvestments, cashAndShortTermInvestments, netReceivables, inventory, otherCurrentAssets, totalCurrentAssets, propertyPlantEquipmentNet, goodwill, intangibleAssets, goodwillAndIntangibleAssets, longTermInvestments, taxAssets, otherNonCurrentAssets, totalNonCurrentAssets, otherAssets, totalAssets, accountPayables, shortTermDebt, taxPayables, deferredRevenue, otherCurrentLiabilities, totalCurrentLiabilities, longTermDebt, deferredRevenueNonCurrent, deferredTaxLiabilitiesNonCurrent,

### Rating Changes

In [86]:
# Variable for rating on previous fixed_quarter_date
# Sort by ticker, fixed_quarter_date
merged = merged.sort_values(by=['ticker', 'fixed_quarter_date'])
# Shift the rating by 1
merged['rating_on_previous_fixed_quarter_date'] = merged.groupby('ticker')['Rating'].shift(1)
merged[['ticker', 'fixed_quarter_date', 'Rating', 'rating_on_previous_fixed_quarter_date']].head(10)

,ticker,fixed_quarter_date,Rating,rating_on_previous_fixed_quarter_date
0,AAPL,2014-07-01 00:00:00.000000,AA,NaN
1,AAPL,2014-10-01 00:00:00.000000,AA,AA
2,AAPL,2015-01-01 00:00:00.000000,AA,AA
3,AAPL,2015-04-01 00:00:00.000000,AA,AA
4,AAPL,2015-07-01 00:00:00.000000,AA,AA
5,AAPL,2015-10-01 00:00:00.000000,AA,AA
6,AAPL,2016-01-01 00:00:00.000000,AA,AA
7,AAPL,2016-04-01 00:00:00.000000,AA,AA
8,AAPL,2016-07-01 00:00:00.000000,AA,AA
9,AAPL,2016-10-01 00:00:00.000000,AA,AA


In [87]:
# Rating change variables
# Get numeric values for Rating and rating_on_next_earnings_call_date
# Encode Ratings as ordinal
rating_order = ['AAA', 'AA', 'A', 'BBB', 'BB', 'B', 'CCC', 'CC', 'C', 'D']
# Reverse
rating_order = rating_order[::-1]
# Create dictionary mapping to number values
rating_dict = {rating: i for i, rating in enumerate(rating_order, 1)}
# Create new column "Rating Rank AAA is 10"
merged['Rating Rank AAA is 10'] = merged['Rating'].map(rating_dict)
# Flag for investment grade - Rating Rank >= 7.0
merged['Investment_Grade'] = merged['Rating Rank AAA is 10'] >= 7.0
# Create new column "rating_on_previous_fixed_quarter_date AAA is 10"
merged['rating_on_previous_fixed_quarter_date AAA is 10'] = merged['rating_on_previous_fixed_quarter_date'].map(rating_dict)

# Cross tab Rating and 'Rating Rank AAA is 10'
print(pd.crosstab(merged['Rating'], merged['Rating Rank AAA is 10']))
# Cross tab rating_on_previous_fixed_quarter_date and 'rating_on_previous_fixed_quarter_date AAA is 10'
print(pd.crosstab(merged['rating_on_previous_fixed_quarter_date'], merged['rating_on_previous_fixed_quarter_date AAA is 10']))

Rating Rank AAA is 10  1   2   3    4     5     6     7     8    9    10
Rating                                                                  
A                       0   0   0    0     0     0     0  1389    0    0
AA                      0   0   0    0     0     0     0     0  339    0
AAA                     0   0   0    0     0     0     0     0    0  137
B                       0   0   0    0  1142     0     0     0    0    0
BB                      0   0   0    0     0  1870     0     0    0    0
BBB                     0   0   0    0     0     0  2233     0    0    0
C                       0  16   0    0     0     0     0     0    0    0
CC                      0   0  14    0     0     0     0     0    0    0
CCC                     0   0   0  185     0     0     0     0    0    0
D                       9   0   0    0     0     0     0     0    0    0
rating_on_previous_fixed_quarter_date AAA is 10  1.0   2.0   3.0   4.0   5.0   \
rating_on_previous_fixed_quarter_date      

In [88]:
# Variable for change before next call
def change(row):
    if row['Rating Rank AAA is 10'] < row['rating_on_previous_fixed_quarter_date AAA is 10']:
        return 'Downgrade Since Last Fixed Quarter Date'
    elif row['Rating Rank AAA is 10'] > row['rating_on_previous_fixed_quarter_date AAA is 10']:
        return 'Upgrade Since Last Fixed Quarter Date'
    elif row['Rating Rank AAA is 10'] == row['rating_on_previous_fixed_quarter_date AAA is 10']:
        return 'Same As Last Fixed Quarter Date'
    else:
        return None

# Apply function to create new column
merged['Change Direction Since Last Fixed Quarter Date'] = merged.apply(change, axis = 1)

# Also numeric version 'Change Since Last Fixed Quarter Date'
merged['Change Since Last Fixed Quarter Date'] = merged['Rating Rank AAA is 10'] - merged['rating_on_previous_fixed_quarter_date AAA is 10']

# Cross tab 'Change Direction Since Last Fixed Quarter Date' and 'Change Since Last Fixed Quarter Date'
print(pd.crosstab(merged['Change Direction Since Last Fixed Quarter Date'], merged['Change Since Last Fixed Quarter Date']))

Change Since Last Fixed Quarter Date            -2.0  -1.0   0.0   1.0   2.0
Change Direction Since Last Fixed Quarter Date                              
Downgrade Since Last Fixed Quarter Date           17   149     0     0     0
Same As Last Fixed Quarter Date                    0     0  6433     0     0
Upgrade Since Last Fixed Quarter Date              0     0     0   180    19


### Sector and Other Textual Company Information

From Kaggle: https://www.kaggle.com/datasets/aramacus/usa-public-companies

Supplemented with manually looked up sector data for missing sectors

In [89]:
# Load CSV 'C:\Users\ijyli\Box\STAT 222 Capstone\Intermediate Data\Sectors\combined_sector_data.csv'
combined_sector_data = pd.read_csv(r'~\Box\STAT 222 Capstone\Intermediate Data\Sectors\combined_sector_data.csv')
print(combined_sector_data)

     Ticker                                        Description  \
0      CTVA  Corteva, Inc. is a global provider of seed and...   
1      ALCO  Alico, Inc. is an agribusiness and land manage...   
2      LMNR  Limoneira Company is primarily an agribusiness...   
3      SANW  S&W Seed Company (S&W) is a multi-crop and mid...   
4       TRC  Tejon Ranch Co. is a diversified real estate d...   
...     ...                                                ...   
3384   RELX                                                NaN   
3385    TAL                                                NaN   
3386   TRTN                                                NaN   
3387   VLRS                                                NaN   
3388    XIN                                                NaN   

        Company Name                  Sector            Industry Group  \
0        CORTEVA INC               Materials                 Materials   
1          ALICO INC        Consumer Staples  Food, Beverag

In [90]:
# Left join us_stocks with merged
merged = pd.merge(merged, combined_sector_data, left_on='ticker', right_on='Ticker', how='left')
print(merged)

     ticker          fixed_quarter_date earnings_call_date Rating  \
0      AAPL  2014-07-01 00:00:00.000000         2014-04-23     AA   
1      AAPL  2014-10-01 00:00:00.000000         2014-07-22     AA   
2      AAPL  2015-01-01 00:00:00.000000         2014-10-20     AA   
3      AAPL  2015-04-01 00:00:00.000000         2015-01-27     AA   
4      AAPL  2015-07-01 00:00:00.000000         2015-04-27     AA   
...     ...                         ...                ...    ...   
7329    ZTS  2015-10-01 00:00:00.000000         2015-08-04    BBB   
7330    ZTS  2016-01-01 00:00:00.000000         2015-11-03    BBB   
7331    ZTS  2016-04-01 00:00:00.000000         2016-02-16    BBB   
7332    ZTS  2016-07-01 00:00:00.000000         2016-05-04    BBB   
7333    ZTS  2016-10-01 00:00:00.000000         2016-08-03    BBB   

                      Rating Agency Name rating_date         Source  \
0     Standard & Poor's Ratings Services  2014-05-27  Supplementary   
1     Standard & Poor's Ratin

In [91]:
# check for duplicates on ticker by fixed_quarter_date
print(merged[merged.duplicated(subset=['ticker', 'fixed_quarter_date'], keep=False)])

Empty DataFrame
Columns: [ticker, fixed_quarter_date, earnings_call_date, Rating, Rating Agency Name, rating_date, Source, Rating Rank AAA is 10, Next Rating, Next Rating Date, Previous Rating, Previous Rating Date, next_rating_date_or_end_of_data, credit_rating_year, previous_fixed_quarter_date, days_since_call, company, quarter, year, earnings_call_datetime, transcript, source, web, date_x, date_y, symbol, reportedCurrency, cik, fillingDate, acceptedDate, calendarYear, period, cashAndCashEquivalents, shortTermInvestments, cashAndShortTermInvestments, netReceivables, inventory, otherCurrentAssets, totalCurrentAssets, propertyPlantEquipmentNet, goodwill, intangibleAssets, goodwillAndIntangibleAssets, longTermInvestments, taxAssets, otherNonCurrentAssets, totalNonCurrentAssets, otherAssets, totalAssets, accountPayables, shortTermDebt, taxPayables, deferredRevenue, otherCurrentLiabilities, totalCurrentLiabilities, longTermDebt, deferredRevenueNonCurrent, deferredTaxLiabilitiesNonCurrent,

### Some Basic Stats

In [92]:
# Summarize columns
print(merged.info())
# print column names
for col in merged.columns:
    print(col)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7334 entries, 0 to 7333
Columns: 171 entries, ticker to Comment
dtypes: bool(2), datetime64[ns](2), float64(114), int32(1), int64(9), object(43)
memory usage: 9.4+ MB
None
ticker
fixed_quarter_date
earnings_call_date
Rating
Rating Agency Name
rating_date
Source
Rating Rank AAA is 10
Next Rating
Next Rating Date
Previous Rating
Previous Rating Date
next_rating_date_or_end_of_data
credit_rating_year
previous_fixed_quarter_date
days_since_call
company
quarter
year
earnings_call_datetime
transcript
source
web
date_x
date_y
symbol
reportedCurrency
cik
fillingDate
acceptedDate
calendarYear
period
cashAndCashEquivalents
shortTermInvestments
cashAndShortTermInvestments
netReceivables
inventory
otherCurrentAssets
totalCurrentAssets
propertyPlantEquipmentNet
goodwill
intangibleAssets
goodwillAndIntangibleAssets
longTermInvestments
taxAssets
otherNonCurrentAssets
totalNonCurrentAssets
otherAssets
totalAssets
accountPayables
shortTermDebt
taxPayabl

In [93]:
# Save a parquet file as well
merged.to_parquet(r'~\Box\STAT 222 Capstone\Intermediate Data\All_Data\all_data_fixed_quarter_dates.parquet', index=False)

In [94]:
# Sample 100 rows to create all_data_sample.csv
if not DEBUG:
    all_data_sample = merged.sample(100)
    all_data_sample.to_csv(r'~\Box\STAT 222 Capstone\Intermediate Data\All_Data\all_data_fixed_quarter_dates_sample.csv', index=False)
    # also save to xlsx
    all_data_sample.to_excel(r'~\Box\STAT 222 Capstone\Intermediate Data\All_Data\all_data_fixed_quarter_dates_sample.xlsx', index=False)

## Fragmented Version to Store on GitHub

In [95]:
# Split dataset into pieces
num_pieces = 8
total_len_pieces = 0
# Delete previous pieces, all contents of '../../../Data/All_Data/All_Data_Fixed_Quarter_Dates'
import os
import shutil
folder = r'../../../Data/All_Data/All_Data_Fixed_Quarter_Dates'
for filename in os.listdir(folder):
    file_path = os.path.join(folder, filename)
    try:
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.unlink(file_path)
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)
    except Exception as e:
        print('Failed to delete %s. Reason: %s' % (file_path, e))
# Save pieces
if not DEBUG:
    for i in range(num_pieces):
        # start index for piece rows
        start_index = i * len(merged) // num_pieces
        # end index for piece rows
        end_index = (i + 1) * len(merged) // num_pieces
        # get piece
        piece = merged[start_index:end_index]
        piece.to_parquet(r'../../../Data/All_Data/All_Data_Fixed_Quarter_Dates/all_data_fixed_quarter_dates_piece_' + str(i) + '.parquet', index=False)
        print(piece.head(5))
        print(len(piece))
        total_len_pieces += len(piece)


  ticker          fixed_quarter_date earnings_call_date Rating  \
0   AAPL  2014-07-01 00:00:00.000000         2014-04-23     AA   
1   AAPL  2014-10-01 00:00:00.000000         2014-07-22     AA   
2   AAPL  2015-01-01 00:00:00.000000         2014-10-20     AA   
3   AAPL  2015-04-01 00:00:00.000000         2015-01-27     AA   
4   AAPL  2015-07-01 00:00:00.000000         2015-04-27     AA   

                   Rating Agency Name rating_date         Source  \
0  Standard & Poor's Ratings Services  2014-05-27  Supplementary   
1  Standard & Poor's Ratings Services  2014-05-27  Supplementary   
2  Standard & Poor's Ratings Services  2014-05-27  Supplementary   
3  Standard & Poor's Ratings Services  2015-02-18  Supplementary   
4  Standard & Poor's Ratings Services  2015-06-02  Supplementary   

   Rating Rank AAA is 10 Next Rating Next Rating Date  ...  \
0                      9          AA       2015-02-18  ...   
1                      9          AA       2015-02-18  ...   
2       

In [96]:
# check total piece length and length of merged
print(total_len_pieces)
print(len(merged))

7334
7334
